In [ ]:
import numpy as np
from sklearn.mixture import GaussianMixture
import librosa
from sklearn.preprocessing import MinMaxScaler

In [ ]:
def getMfcc(wav, i):
    left = len(wav)//2 - len(wav)//4
    right = len(wav)//2 + len(wav)//4
    
    wav = wav[left:right]
    
    wav = librosa.util.fix_length(wav, size=16000)
    wav = librosa.effects.preemphasis(y=wav, coef=0.97)

    mfcc = librosa.feature.mfcc(y=wav,
                                sr=16000,
                                n_fft=400,
                                hop_length=160,
                                n_mfcc=i
                                )
    
    mfcc = mfcc[1:]
    
    scaler = MinMaxScaler()
    mfcc = scaler.fit_transform(mfcc)

    
    return mfcc.T

In [ ]:
train_data_file = "fmcc_train.ctl"

In [ ]:
train_feml = []
train_male = []

# Train 데이터 남/여 구분
with open("data/" + train_data_file, 'r') as file:
    for file_path in file:
        file_path = file_path.rstrip("\n")
        wav, sr = librosa.load("data/wav16k/train/" + file_path + ".wav", sr=16000, mono=True)
        
        mfcc = getMfcc(wav)

        if file_path[0] == "F":
            train_feml.extend(mfcc)
        else:
            train_male.extend(mfcc)

feml = np.array(train_feml)
male = np.array(train_male)

np.save('female.npy', feml)
np.save('male.npy', male)

In [ ]:
male_gmm = GaussianMixture(n_components=1, random_state=42)
female_gmm = GaussianMixture(n_components=1, random_state=42)

In [ ]:
feml = np.load('female.npy')
male = np.load('male.npy')

In [ ]:
male_gmm.fit(male)
female_gmm.fit(feml)

rate = 0

In [8]:

for i in range(50, 150):
    train_feml = []
    train_male = []

    # Train 데이터 남/여 구분
    with open("data/" + train_data_file, 'r') as file:
        for file_path in file:
            file_path = file_path.rstrip("\n")
            wav, sr = librosa.load("data/wav16k/train/" + file_path + ".wav", sr=16000, mono=True)

            mfcc = getMfcc(wav, i)

            if file_path[0] == "F":
                train_feml.extend(mfcc)
            else:
                train_male.extend(mfcc)

    feml = np.array(train_feml)
    male = np.array(train_male)

    np.save('female.npy', feml)
    np.save('male.npy', male)
    male_gmm = GaussianMixture(n_components=1, random_state=42)
    female_gmm = GaussianMixture(n_components=1, random_state=42)
    feml = np.load('female.npy')
    male = np.load('male.npy')
    male_gmm.fit(male)
    female_gmm.fit(feml)
    rate = 0

    with open("data/fmcc_test_ref.txt", 'r') as file:
        for file_path in file:
            file_path = file_path.rstrip("\n")

            file_path, gender = file_path.split()

            wav, frame_rate = librosa.load("data/wav16k/test/" + file_path + ".wav",
                                           sr=16000, mono=True)
            mfcc = getMfcc(wav, i)

            male_score = male_gmm.score(mfcc)
            female_score = female_gmm.score(mfcc)

            if gender == "male":
                if male_score >= female_score:
                    rate += 1
            else:
                if male_score <= female_score:
                    rate += 1

    print(str(i) + "번째")
    print(rate/9)
    print(rate)
    print("---------------------------------")

50번째
90.55555555555556
815
---------------------------------
51번째
90.55555555555556
815
---------------------------------
52번째
90.66666666666667
816
---------------------------------
53번째
91.11111111111111
820
---------------------------------
54번째
91.0
819
---------------------------------
55번째
91.33333333333333
822
---------------------------------
56번째
91.33333333333333
822
---------------------------------
57번째
91.22222222222223
821
---------------------------------
58번째
91.44444444444444
823
---------------------------------
59번째
91.44444444444444
823
---------------------------------
60번째
91.44444444444444
823
---------------------------------
61번째
91.22222222222223
821
---------------------------------
62번째
91.55555555555556
824
---------------------------------
63번째
91.77777777777777
826
---------------------------------
64번째
91.77777777777777
826
---------------------------------
65번째
91.66666666666667
825
---------------------------------
66번째
91.88888888888889
827
----------